In [1]:
%%bash
pip install kafka-python

In [2]:
from kafka import KafkaConsumer
import json
import csv
import re

In [3]:
consumer = KafkaConsumer(
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    value_deserializer=lambda m: json.loads(m.decode('ascii','ignore'))
)


In [4]:
consumer.subscribe('jaeger-spans')
f = open('/home/vmadmin/Log/ml/log generation/collected_logs.csv', 'a', encoding='utf-8')
header_writer = csv.writer(f)
header_writer.writerow(["duration","http_status_code","comment"])
f.close()

In [ ]:
counter = 1
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    # print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition, message.offset, message.key, message.value))
    
    if message.value['process']['serviceName'] == 'frontend':
        
        # open the file in the append mode
        f1 = open('/home/vmadmin/Log/ml/log generation/collected_logs.csv', 'a', encoding='utf-8')
        # create the csv writer
        writer = csv.writer(f1)
        
        http_method = re.findall(r"{'key': 'http\.method', 'vStr': '(.+?)'}",str(message.value))
        http_target = re.findall(r"{'key': 'http\.target', 'vStr': '(.+?)'}",str(message.value))
        http_status_code = re.findall(r"{'key': 'http\.status_code', 'vType': '.*?', 'vInt64': '(\d+)'}",str(message.value))
        http_url = re.findall(r"{'key': 'http\.url', 'vStr': '(.+?)'}",str(message.value))
        duration = re.findall(r"(.+?)s",str(message.value['duration']))
        
        if len(http_method) > 0:
            http_method = http_method[0]
        if len(http_target) > 0:
            http_target = http_target[0]
        if len(http_url) > 0:
            http_url = http_url[0]
        if len(http_status_code) > 0:
            http_status_code = http_status_code[0]
        if len(duration) > 0:
            duration = duration[0]
        
        if http_method == 'POST':
            comment = ""
            fields = message.value['logs'][0]['fields']
            description1 = re.findall(r"{'key': 'body', 'vStr': '(.+?)'}",str(fields))
            if len(description1) > 0:
                temp = re.findall(r'"description":"(.+?)","_links":',str(description1[0]))
                if len(temp) > 0:
                    comment = temp[0]
            
            data = [duration,http_status_code,comment]
            
            # write a row to the csv file
            writer.writerow(data)
            
            print(counter)
            counter = counter + 1

        # close the file
        f1.close()

17
18
19
20
21
22
23
24
25
26
27
